In [87]:
import pandas as pd
import csv
import string

In [5]:
file_path="tweets.tsv"

In [6]:
tweets_df=pd.read_csv(file_path,delimiter='\t')

In [148]:
tweets_df['post'] = tweets_df['post'].str.lower()
tweets_df['post'] = tweets_df['post'].str.strip()
def remove_punctuation(text):
    # Create a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Apply the remove_punctuation function to the "text" column
tweets_df['post'] =tweets_df['post'].apply(remove_punctuation)

In [149]:
tweets_df['post']

0        jewish harvard professor noel ignatiev wants t...
1        bhigher education is a part of european cultur...
2        has a problem with   the whites   and   the ch...
3        is yasir qadhi a hate preacher for calling chr...
4        rt    how three million germans mass murdered ...
                               ...                        
21475    seeing prostitutes is morally wrong but being ...
21476    i wonder how many females i raped today i spen...
21477    having a criminal record is more attractive to...
21478    another advantage of being a female getting by...
21479    if men are so privileged and women have it so ...
Name: post, Length: 21480, dtype: object

In [123]:
# Load hashtags from Excel file
hashtags_df = pd.read_excel("hashtags.xlsx")
hashtags_df=hashtags_df.head(100)
hashtags_list = hashtags_df['Normalized Hashtags'].tolist()


In [124]:
hashtags_df

,Hashtag,Normalized Hashtags,One example in the dataset,Explain,"Confident? If not, why"
0,#nazis,#nazis,NaN,NaN,NaN
1,#immigration's,#immigration,NaN,NaN,NaN
2,#moonbats,#moonbats,NaN,NaN,NaN
3,#key,#key,NaN,NaN,NaN
4,#hhsmandate,#hhsmandate,NaN,NaN,NaN
...,...,...,...,...,...
95,#liberty,#liberty,NaN,NaN,NaN
96,#moscow,#moscow,NaN,NaN,NaN
97,#handoverlakhvi,#handoverlakhvi,NaN,NaN,NaN
98,#hypocrisy,#hypocrisy,NaN,NaN,NaN


In [90]:
pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.1/371.1 kB 6.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [102]:
import openai

api_key = "Enter API key"

In [125]:
explanations=[]
for hashtag in hashtags_list:
    prompt = f"Give a summary regarding this {hashtag} hashtag in 1 sentence"
    response = openai.Completion.create(
        engine="text-davinci-002",  # Choose the appropriate GPT-3 engine
        prompt=prompt,
        max_tokens=120,  # Adjust as needed
        api_key=api_key
        )
    generated_text = response.choices[0].text
    explanations.append(generated_text)


In [135]:
explanations = [s.replace('\n'*s.count('\n'), '') for s in explanations]

In [136]:
explanations

['The #nazis hashtag has been used to share information and galvanize opposition to the Nazi party and its ideals.',
 'The#immigration hashtag is being used to discuss the current immigration situation in the United States.',
 'The #moonbats hashtag is a derogatory term used to describe people who are considered to be liberal or left-wing.',
 'The #key hashtag can be used for a variety of purposes, such as finding a key for a lost or stolen item, or sharing a key for a locked location.',
 ".The #hhsmandate hashtag was created to provide a space for people to share their stories and experiences with the US Department of Health and Human Services' (HHS) mandate that religious organizations must provide birth control and other forms of contraception to employees.",
 'The #secede hashtag is being used by people who want the southern states to secede from the United States.',
 'The hashtag #religiousfreedomundeniable is used to underscore the importance of religious freedom and to point out

,Hashtag,Explain
0,#nazis,The #nazis hashtag has been used to share info...
1,#immigration,The#immigration hashtag is being used to discu...
2,#moonbats,The #moonbats hashtag is a derogatory term use...
3,#key,The #key hashtag can be used for a variety of ...
4,#hhsmandate,.The #hhsmandate hashtag was created to provid...


In [141]:
hashtags_df.to_excel("updated_hashtags_with_explanation.xlsx", index=False)

In [153]:
tweet_text=tweets_df['post'].tolist()

In [181]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the hashtag descriptions and tweet text
tfidf_matrix = tfidf_vectorizer.fit_transform(explanations + tweet_text)

# Calculate cosine similarity between the hashtag descriptions and tweet text
cosine_sim = linear_kernel(tfidf_matrix[:len(explanations)], tfidf_matrix[len(explanations):])

# Find the index of the tweet with the highest similarity score for each hashtag
closest_tweet_indices = cosine_sim.argmax(axis=1)
# Retrieve the closest tweet for each hashtag
closest_tweets = [tweet_text[i] for i in closest_tweet_indices]

In [205]:
confidence=[]
for i, hashtag in enumerate(hashtags_list):
    similarity_score = cosine_sim[i][0]*100  # Assuming there's only one closest tweet

    if similarity_score > 0.8:
        confidence.append("Yes")
    else:
        confidence.append("No")

In [206]:
hashtags_df = pd.DataFrame({'Hashtag': hashtags_list,'Closest_tweet':closest_tweets, 'Explain': explanations,'Confidence':confidence})

In [209]:
hashtags_df

,Hashtag,Closest_tweet,Explain,Confidence
0,#nazis,american ideals are actually new england ideal...,The #nazis hashtag has been used to share info...,Yes
1,#immigration,they arent us white people are to blame for o...,The#immigration hashtag is being used to discu...,Yes
2,#moonbats,right you used a more palatable less scary t...,The #moonbats hashtag is a derogatory term use...,Yes
3,#key,if spencer shows the media coverage will go na...,The #key hashtag can be used for a variety of ...,No
4,#hhsmandate,more companies suing obama for hhs mandate vio...,.The #hhsmandate hashtag was created to provid...,Yes
...,...,...,...,...
95,#liberty,conservatism is dead were going to have white...,The #liberty hashtag is used by people who are...,Yes
96,#moscow,racist tweets,The current top tweets with #moscow are mostly...,Yes
97,#handoverlakhvi,national socialist party pays homage to all de...,The goal of the #handoverlakhvi hashtag is to ...,Yes
98,#hypocrisy,or we could continue to point out antifa hypoc...,.The #hypocrisy hashtag is used to point out t...,Yes


In [208]:
hashtags_df.to_excel('result.xlsx',index=False)